In [1]:
""" Use the vegetation survey data to calculate the percentage cover of plant functional types"""
import pandas as pd
import os
import numpy as np

path_root = os.getcwd()
path_data = os.path.join(path_root, "..", "data", "Hubbard Brook")

survey_files = {
    1996: os.path.join(path_data, "knb-lter-hbr.40.7", "w1_1996veg.txt"), 
    2001: os.path.join(path_data, "knb-lter-hbr.41.7", "w1_2001veg.txt"), 
    2006: os.path.join(path_data, "knb-lter-hbr.142.3", "w1_2006veg.txt"),
    2011: os.path.join(path_data, "knb-lter-hbr.143.3", "w1_2011veg.txt"),
    2016: os.path.join(path_data, "knb-lter-hbr.240.2", "w1_2016_vegInventory.csv"),
}

In [2]:
basal_area_summary = {}

for year in survey_files.keys():
    data = pd.read_csv(survey_files[year])

    # Dbh: cm, Basal area: m^2
    data['Basal area'] = np.pi * (data['Dbh']/200)**2

    # Get the PFTs
    sp_to_pft = pd.read_excel("Hubbard Brook species to PFT table.xlsx", sheet_name = "Sheet1")
    abbr_to_pft = {a:int(b) for a,b in zip(sp_to_pft['Abbreviation'], sp_to_pft['PFT number']) if not np.isnan(b)}
    data['PFT'] = data['Species'].map(abbr_to_pft)

    def process_one_plot(df):
        # Calculate the relative basal area of the various species, 
        # multiply by the area of the plot
        tot_ba = df.groupby('PFT').sum()['Basal area']
        rel_ba = tot_ba / tot_ba.sum()
        scaled_ba = rel_ba * df['10Area'].iloc[0]
        return scaled_ba

    # Sum up the plots and rescale
    tot_ba = data.groupby('Plot').apply(process_one_plot).groupby('PFT').sum()
    rel_ba = tot_ba / tot_ba.sum() * 100.

    basal_area_summary[year] = rel_ba
basal_area_summary = pd.DataFrame(basal_area_summary)

In [3]:
basal_area_summary

,1996,2001,2006,2011,2016
PFT,,,,,
2.0,5.502716,6.105590,6.537898,7.239026,7.258774
7.0,90.727709,89.239103,90.023317,89.825016,90.631124
8.0,3.769576,4.652014,3.438785,2.935957,2.110102
10.0,NaN,0.003293,NaN,NaN,NaN


In [7]:
basal_area_summary.mean(axis = 1).round(2)

PFT
2.0      6.53
7.0     90.09
8.0      3.38
10.0     0.00
dtype: float64